In [ ]:
"""!pip install ffn
!pip install pypfopt
!pip install empyrical"""

In [ ]:
pip install seaborn

In [ ]:
import ffn

import numpy as np
import pandas as pd
#from empyrical import alpha_beta
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
#import matplotlib.pyplot as plt
import warnings
#plt.style.use('dark_background')
#warnings.simplefilter(action='ignore', category=FutureWarning)

%matplotlib inline

In [ ]:

folder = '../data/'
file = 'ffn.csv'
filename = f"{folder}{file}"
isin='LU0843168575'
filename

isins=['LU0280438135','LU0255977372','LU0133806785']

In [ ]:
prices = ffn.get(f'{isins[0]}-NAV,{isins[1]}-NAV,{isins[2]}-NAV', provider=ffn.data.csv, path=filename)
prices


In [ ]:
sns.lineplot(data=prices)
plt.show()

In [ ]:
#prices = ffn.get('TTWO, CCOEY, EA', start='2019-01-01')
#benchmark = ffn.get(f'{isins[0]}-level',provider=ffn.data.csv, path=filename)
#ax = prices.rebase().plot(figsize=(12,5))
#plt.show()

In [ ]:
prices

In [ ]:
returns = prices['2007-01-31':'2022-12-31'].to_returns().dropna()

In [ ]:
returns.plot()
plt.show()

In [ ]:
stats = prices['2007-01-31':'2021-10-31'].calc_stats()
stats.display()

In [ ]:
stats.prices.to_drawdown_series().plot(figsize=(12,5))
plt.show()

In [ ]:
stats.prices.calc_cagr()

In [ ]:
returns = prices.to_log_returns().dropna()
returns.calc_mean_var_weights().as_format('.2%')

In [ ]:
returns.calc_erc_weights().as_format('.2%')

In [ ]:
from empyrical import alpha_beta

symbols = [f'{isin}-NAV']
for sym in symbols:
    stk = ffn.get(sym, provider=ffn.data.csv, path=filename)
    alpha, beta = alpha_beta(stk, benchmark)
    print(sym, beta)


In [ ]:
returns = prices.pct_change()
weight = np.array([10,10,10])

t = np.sum(weight)
weights = weight/t
# mean daily return and covariance of daily returns
mean_daily_returns = returns.mean()
cov_matrix = returns.cov()

portfolio_return = round(np.sum(mean_daily_returns * weights) * 252,2)
portfolio_std_dev = round(np.sqrt(np.dot(weights.T,np.dot(cov_matrix, weights))) * np.sqrt(252),2)

print("Expected annualised return: " + str(portfolio_return*100))
print("Volatility: " + str(portfolio_std_dev*100))

In [ ]:
portfolio_return

In [ ]:
mean_daily_returns

In [ ]:
returns

In [ ]:
portfolio_return

In [ ]:
prices.iloc[0]

In [ ]:
#pip install PyPortfolioOpt

In [ ]:
import pyfolio as pf

In [ ]:
prices

In [ ]:
x=ffn.core.drawdown_details(ffn.core.calc_max_drawdown(prices))

In [ ]:
ffn.core.GroupStats(prices).display()

In [ ]:
100*((1*(106.56/100))+(1*(229.62/100))+(1*(80.63/100))-3)

In [ ]:
(((1*(106.56/100)))+1)+(((1*(229.62/100)))+1)+(((1*(80.63/100)))+1)-3


In [ ]:
(((1*(229.62/100)))+1)/3

In [ ]:
((1*(80.63/100)))+1

In [ ]:
pf?

In [ ]:
prices

In [ ]:
returns 

In [ ]:
p = prices.copy()

In [ ]:
p

In [ ]:
p.index = p.index.tz_localize('utc')

In [ ]:
p

In [ ]:
returns = p.lu0280438135nav.pct_change()

In [ ]:
pf.create_returns_tear_sheet(returns, live_start_date='2007-01-31')

In [2]:
(-23.91 - 11.36 - 20.61)/3

-18.626666666666665

In [3]:
(-19.77 - 2.30 - 5.34) / 3

-9.136666666666667